In [1]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
#sns.set_style('whitegrid')
import h5py
from PIL import Image
import os
from tqdm import tqdm_notebook as tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [2]:
cd ..

/Users/james/Desktop/Projects/ml_strategy


In [3]:
import tushare as ts
#ts.set_token('') #one time only
pro = ts.pro_api()

In [ ]:
df = pro.trade_cal(exchange='', start_date='20180901', end_date='20181001', \
                   fields='exchange,cal_date,is_open,pretrade_date', is_open='0')
#df1 = pro.query('trade_cal', exchange='', start_date='20180901', end_date='20181001',\
#               fields='exchange,cal_date,is_open,pretrade_date', is_open='0')

In [4]:
# 确定起止日期
start_date = '2016-01-01 00:00:00'
end_date = '2019-12-20 23:59:59'

In [5]:
# 做一个用于tushare调用的list
from dateutil import rrule
from datetime import datetime

all_dates_list = []
for dt in rrule.rrule(rrule.DAILY,
                      dtstart=datetime.strptime(start_date.split(' ')[0], '%Y-%m-%d'),
                      until=datetime.strptime(end_date.split(' ')[0], '%Y-%m-%d')):
    all_dates_list.append(dt.strftime('%Y%m%d'))

# twitter

In [28]:
twitter_news = pro.exchange_twitter(start_date=all_dates_list[-500]+' 00:00:00', \
                                    end_date=all_dates_list[-1]+' 23:59:59', \
                     fields="id,account,nickname,content,retweet_content,media,str_posted_at,create_at")

In [15]:
for i in tqdm(all_dates_list):
    if not os.path.exists('data/news/twitternews/'+i+'.csv'):
        df = pro.exchange_twitter(start_date=i+' 00:00:00', end_date=i+' 23:59:59', \
                     fields="id,account,nickname,content,retweet_content,media,str_posted_at,create_at")
        df.to_csv('data/news/twitternews/'+i+'.csv')
        #time.sleep(1) #每分钟最多访问该接口2次
    

KeyboardInterrupt: 

In [16]:
df

,id,account,nickname,content,retweet_content,media,str_posted_at,create_at


# 交易所
似乎无用

In [30]:
df = pro.exchange_ann(start_date=start_date, end_date=end_date, \
                fields='title, datetime')

# CCTV

In [35]:
for i in tqdm(all_dates_list):
    if not os.path.exists('data/news/cctv/'+i+'.csv'):
        pro.cctv_news(date=i).to_csv('data/news/cctv/'+i+'.csv')
        time.sleep(0.013)
    

# 新闻通讯
单次最大60行

In [ ]:
for i in tqdm(all_dates_list):
    if not os.path.exists('data/news/majornews/'+i+'.csv'):
        df = pro.major_news(src='', start_date=i+' 00:00:00', \
                    end_date=i+' 23:59:59', fields='title,content')
        df.to_csv('data/news/majornews/'+i+'.csv')
        time.sleep(31) #每分钟最多访问该接口2次
    

# 新闻快讯
多种来源 src
新浪财经	sina	获取新浪财经实时资讯
华尔街见闻	wallstreetcn	华尔街见闻快讯
同花顺	10jqka	同花顺财经新闻
东方财富	eastmoney	东方财富财经新闻
云财经	yuncaijing	云财经新闻

In [8]:
newssrc = ['sina','wallstreetcn','10jqka','eastmoney','yuncaijing']

In [9]:
for i in tqdm(range(len(all_dates_list))):
    for src in newssrc:
        if not os.path.exists('data/news/news/'+all_dates_list[i]+src+'.csv'):
            df = pro.news(src=src, start_date=all_dates_list[i], \
                        end_date=all_dates_list[i+1])
            df.to_csv('data/news/news/'+all_dates_list[i]+src+'.csv')
            time.sleep(7) #每分钟最多访问该接口10次

ReadTimeout: HTTPConnectionPool(host='api.waditu.com', port=80): Read timed out. (read timeout=15)

# 上市公司信息
为了接续读取所有数据，从第一天开始，然后如果满50再从最后的那天开始继续，最终merge 表格

In [86]:
allcompany = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
startdate = all_dates_list[0]
enddate=all_dates_list[-1]

for i in tqdm(range(len(allcompany))):
    if not os.path.exists('data/news/company/'+allcompany['ts_code'][i]+'.csv'):
        singlestockalldf = pro.anns(ts_code=allcompany['ts_code'][i], start_date=startdate, end_date=enddate)
        if singlestockalldf.shape[0] ==50:
            #print ('new')
            startdate = np.array(df.ann_date)[-1] #表格最后一天
            df = pro.anns(ts_code=allcompany['ts_code'][i], start_date=startdate, end_date=enddate)
            singlestockalldf = pd.concat((singlestockalldf,df),axis=0)
        singlestockalldf.to_csv('data/news/company/'+allcompany['ts_code'][i]+'.csv')
        time.sleep(0) #每分钟最多访问该接口10次

ReadTimeout: HTTPConnectionPool(host='api.waditu.com', port=80): Read timed out. (read timeout=15)

In [76]:
len(allcompany)

3750

In [39]:
df = pro.anns(ts_code='002149.SZ', start_date=start_date.split(' ')[0], end_date=end_date.split(' ')[0])

In [57]:
df = pro.anns(ts_code='002149.SZ', start_date=all_dates_list[0], end_date=all_dates_list[-1])

new


In [74]:
singlestockalldf

,ts_code,ann_date,title,content
0,002149.SZ,20191116,2019年第三次临时股东大会的法律意见书,\n国浩律师（西安）事务所 \n关于西部金属材料股份有限公司 \n2019年第三次临时...
1,002149.SZ,20191116,第七届董事会第一次会议决议公告,\n证券代码：002149 证券简称：西部材料 公告编号：20...
2,002149.SZ,20191116,2019年第三次临时股东大会决议公告,1 \n证券代码：002149 证券简称：西部材料 公告编号：...
3,002149.SZ,20191116,第七届监事会第一次会议决议公告,证券代码：002149 证券简称：西部材料 公告编号：2019-...
4,002149.SZ,20191116,独立董事对公司相关事项的独立意见,西部金属材料股份有限公司 \n独立董事对公司相关事项的独立意见 \n \n根据《公司章程》...
5,002149.SZ,20191031,关于董事会换届选举的公告,证券代码：002149 证券简称：西部材料 公告编号：...
6,002149.SZ,20191031,关于调整部分募集资金投资项目规模并将剩余募集资金永久补充流动资金的公告,证券代码：002149 证券简称：西部材料 公告编号：2...
7,002149.SZ,20191031,九州证券股份有限公司关于公司调整部分募投项目规模并将剩余募集资金永久补充流动资金的核查意见,\n \n九州证券股份有限公司 \n关于西部金属材料股份有限公司 \n调整部分募投项目规模...
8,002149.SZ,20191031,变更募投项目实施方式及关联交易的公告,证券代码：002149 证券简称：西部材料 公告编号：2...
9,002149.SZ,20191031,独立董事候选人声明（郭斌）,西部金属材料股份有限公司 独立董事候选人声明 \n \n声明人 郭斌 ，作为西部金属...


In [ ]:
# 跑twitter majornews  news  allcompany